In [1]:
# Load Packages - KEEP MINIMAL FOR FISHTANK COMPATIBILITY
import pandas as pd
import numpy as np
import os
import sys
import time

In [3]:
# set locations for working files
if len(sys.argv) != 3:
    print("Usage: python3 process-sim.py <automation_dir> <attpcroot_dir>")
    print('Assuming testing directories')
    automation_dir = '/mnt/analysis/e17023/Adam/GADGET2/'
    attpcroot_dir = '/mnt/analysis/e17023/Adam/ATTPCROOTv2/'
else:
    # Automation directory
    automation_dir = sys.argv[1]
    
    # ATTPCROOTv2 directory
    attpcroot_dir = sys.argv[2]

Usage: python iter-params.py <automation_dir> <attpcroot_dir>
Assuming testing directories


In [78]:
def indicator_file(file_type, indicator_directory=automation_dir):
    df = pd.DataFrame([0])
    df.to_csv(indicator_directory + file_type + '.csv', index=False)
    #print(file_type + ' FILE CREATED')

In [80]:
parameters = pd.read_csv(automation_dir + 'simInput/parameters.csv')

In [22]:
# check for and complete any active simulations

# 0 = inactive
# 1 = active
# 2 = complete

active_sims = parameters[parameters['Status'] == 1]

if len(active_sims) > 1:
    indicator_file('STOP')
    raise Exception('More than one simulation marked as active')

# Search for output.h5 and rename
Complete = False
for filename in os.listdir(automation_dir + 'simOutput/'):
    f = os.path.join(automation_dir + 'simOutput/', filename)
    # checking if it is a file
    if os.path.isfile(f):
        if filename == 'output.h5':
            os.rename(f, automation_dir + 'simOutput/hdf5/' + active_sims.loc[active_sims.index[0],'Sim']+'.h5')
            Complete = True

# Set Status in parameters
if Complete:
    parameters.loc[active_sims.index[0], 'Status'] = 2
    #print('Simulation', parameters.loc[active_sims.index[0], 'Sim'] + ' complete')
    if 'time' in parameters.columns:
        parameters.loc[active_sims.index[0], 'time'] = time.time() - parameters.loc[active_sims.index[0], 'time']
else:
    indicator_file('STOP')
    raise Exception('Could not find output file')

STOP FILE CREATED


Exception: Could not find output file

In [ ]:
# Update parameters.csv
parameters.to_csv(automation_dir + 'simInput/parameters.csv', index=False)